In [639]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import myslack
import datetime as dt
import pickle
import time
import re
from tqdm import trange

In [664]:
driver = webdriver.Chrome()  
url = 'http://produce48.mnet.com/pc/profile/0'
driver.get(url) 

In [850]:
review_count = int(driver.find_element_by_xpath('//*[@id="comment"]/div/h2/strong').text)
comment_page_count = int(review_count/20)+1
review_count, comment_page_count

(8493, 425)

In [851]:
def open_all_comment():
    print('이전 댓글 모두 불러오기 시작...')    
    for _ in trange(comment_page_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/a').click()
            time.sleep(0.3)
        except Exception as e:
            print(e)
            time.sleep(1)
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('이전 댓글 모두 불러오기 성공!')

In [852]:
def make_produce48_df(trainee_name):
    print(trainee_name+'의 댓글 수: '+str(review_count))
    open_all_comment()
    
    print('making dataframe...')
    df = pd.DataFrame(columns=['User_ID','Comment','Date'])
    for i in trange(review_count):
        try:
            df.loc[len(df)] = { 
            'User_ID':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/strong'.format(i+1)).text,
            'Comment':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/p'.format(i+1)).text,
            'Date':driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/dl[{}]/dd/span'.format(i+1)).text.split(' ')[0],
            }
        except Exception as e:
            print(e)
            pass
    print('making dataframe done!')
    return df

In [853]:
def make_df_name():
    eng_name = \
    re.sub("[^a-zA-Z]"," ",driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]').text).strip()
    result = eng_name.replace(' ','_')
    return result

name = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/div[1]/dl[2]/dt/p[2]/span').text
agency = driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[2]/h4/span').text

make_df_name(), name, agency

('KANG_HYE_WON', '강혜원', '에잇디')

In [854]:
%time KANG_HYE_WON = make_produce48_df(name)

  0%|          | 0/425 [00:00<?, ?it/s]

강혜원의 댓글 수: 8493
이전 댓글 모두 불러오기 시작...


 96%|█████████▌| 406/425 [35:14<01:38,  5.21s/it]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="comment"]/div/div[2]/div/a"}
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.39.562713 (dd642283e958a93ebf6891600db055f1f1b4f3b2),platform=Mac OS X 10.13.6 x86_64)



  0%|          | 0/8493 [00:00<?, ?it/s]

이전 댓글 모두 불러오기 성공!
making dataframe...


100%|██████████| 8493/8493 [6:34:08<00:00,  2.78s/it]  

making dataframe done!
CPU times: user 1min 2s, sys: 5.35 s, total: 1min 7s
Wall time: 7h 10min 48s


In [855]:
KANG_HYE_WON.shape

(8493, 3)

In [856]:
KANG_HYE_WON.tail()

,User_ID,Comment,Date
8488,권양수,오 괜찮타 예도 안구정화 갑이내,2018-05-12
8489,김윤호,제발 덕후들이라면 강혜원 응원합시다,2018-05-12
8490,서상우,매력 있네요. 예쁘삼. ㅎ,2018-05-12
8491,salsaaabill.l,It is so cute,2018-05-11
8492,강선덕,완전 만찢녀 청순 귀염귀염~,2018-05-11


In [857]:
pickle.dump(KANG_HYE_WON, open('./produce48/에잇디/KANG_HYE_WON.pkl','wb'))

In [858]:
KANG_HYE_WON_df = pickle.load(open('./produce48/에잇디/KANG_HYE_WON.pkl','rb'))

In [859]:
myslack.send_slack('{} finish!'.format(make_df_name()))

<Response [200]>


In [860]:
KANG_HYE_WON_df.tail()

,User_ID,Comment,Date
8488,권양수,오 괜찮타 예도 안구정화 갑이내,2018-05-12
8489,김윤호,제발 덕후들이라면 강혜원 응원합시다,2018-05-12
8490,서상우,매력 있네요. 예쁘삼. ㅎ,2018-05-12
8491,salsaaabill.l,It is so cute,2018-05-11
8492,강선덕,완전 만찢녀 청순 귀염귀염~,2018-05-11


In [861]:
driver.close() # 창닫기
driver.quit() # 브라우져 닫기